In [75]:
from glob import glob
import IPython.display as ipd
import matplotlib.pyplot as plt
from pyAudioAnalysis import audioBasicIO as aIo
from pyAudioAnalysis import ShortTermFeatures as sf
import librosa
import librosa.display
import numpy as np
import pandas as pd
import warnings
warnings.filterwarnings("ignore")

In [141]:
file_paths = glob('../Downloads/Audio-Classification-master/Audio_data/*/*.wav')

In [143]:
import os
path = '../Downloads/Audio-Classification-master/Audio_data/'
neutral= os.listdir(path+'neutral')
angry= os.listdir(path+'angry')
for i in range(len(neutral)):
    neutral[i] = '../Downloads/Audio-Classification-master/Audio_data/neutral/' + neutral[i]
neutral
for i in range(len(angry)):
    angry[i] = '../Downloads/Audio-Classification-master/Audio_data/angry/' + angry[i]
neutral

['../Downloads/Audio-Classification-master/Audio_data/neutral/40Label.wav',
 '../Downloads/Audio-Classification-master/Audio_data/neutral/41Label.wav',
 '../Downloads/Audio-Classification-master/Audio_data/neutral/09Label.wav',
 '../Downloads/Audio-Classification-master/Audio_data/neutral/08Label.wav',
 '../Downloads/Audio-Classification-master/Audio_data/neutral/37Label.wav',
 '../Downloads/Audio-Classification-master/Audio_data/neutral/36Label.wav',
 '../Downloads/Audio-Classification-master/Audio_data/neutral/03Label.wav',
 '../Downloads/Audio-Classification-master/Audio_data/neutral/02Label.wav',
 '../Downloads/Audio-Classification-master/Audio_data/neutral/57Label.wav',
 '../Downloads/Audio-Classification-master/Audio_data/neutral/56Label.wav',
 '../Downloads/Audio-Classification-master/Audio_data/neutral/20Label.wav',
 '../Downloads/Audio-Classification-master/Audio_data/neutral/21Label.wav',
 '../Downloads/Audio-Classification-master/Audio_data/neutral/14Label.wav',
 '../Downloa

In [144]:
emotions=[]
for item in neutral:
    emotions.append("neutral")
for item in angry:
    emotions.append("angry")

In [145]:
emotions

['neutral',
 'neutral',
 'neutral',
 'neutral',
 'neutral',
 'neutral',
 'neutral',
 'neutral',
 'neutral',
 'neutral',
 'neutral',
 'neutral',
 'neutral',
 'neutral',
 'neutral',
 'neutral',
 'neutral',
 'neutral',
 'neutral',
 'neutral',
 'neutral',
 'neutral',
 'neutral',
 'neutral',
 'neutral',
 'neutral',
 'neutral',
 'neutral',
 'neutral',
 'neutral',
 'neutral',
 'neutral',
 'neutral',
 'neutral',
 'neutral',
 'neutral',
 'neutral',
 'neutral',
 'neutral',
 'neutral',
 'neutral',
 'neutral',
 'neutral',
 'neutral',
 'neutral',
 'neutral',
 'neutral',
 'neutral',
 'neutral',
 'neutral',
 'neutral',
 'neutral',
 'neutral',
 'neutral',
 'neutral',
 'neutral',
 'neutral',
 'neutral',
 'neutral',
 'neutral',
 'angry',
 'angry',
 'angry',
 'angry',
 'angry',
 'angry',
 'angry',
 'angry',
 'angry',
 'angry',
 'angry',
 'angry',
 'angry',
 'angry',
 'angry',
 'angry',
 'angry',
 'angry',
 'angry',
 'angry',
 'angry',
 'angry',
 'angry',
 'angry',
 'angry',
 'angry',
 'angry',
 'angry',


In [146]:
paths = neutral+angry
paths

['../Downloads/Audio-Classification-master/Audio_data/neutral/40Label.wav',
 '../Downloads/Audio-Classification-master/Audio_data/neutral/41Label.wav',
 '../Downloads/Audio-Classification-master/Audio_data/neutral/09Label.wav',
 '../Downloads/Audio-Classification-master/Audio_data/neutral/08Label.wav',
 '../Downloads/Audio-Classification-master/Audio_data/neutral/37Label.wav',
 '../Downloads/Audio-Classification-master/Audio_data/neutral/36Label.wav',
 '../Downloads/Audio-Classification-master/Audio_data/neutral/03Label.wav',
 '../Downloads/Audio-Classification-master/Audio_data/neutral/02Label.wav',
 '../Downloads/Audio-Classification-master/Audio_data/neutral/57Label.wav',
 '../Downloads/Audio-Classification-master/Audio_data/neutral/56Label.wav',
 '../Downloads/Audio-Classification-master/Audio_data/neutral/20Label.wav',
 '../Downloads/Audio-Classification-master/Audio_data/neutral/21Label.wav',
 '../Downloads/Audio-Classification-master/Audio_data/neutral/14Label.wav',
 '../Downloa

In [147]:
def load_file(filepath):
    y,sr = librosa.load(filepath)
    return (y,sr)

In [148]:
def divide_segments(y_time,start):
    # divide into 50 ms segments
    # 50ms -> means 1102 samples    
    y_segmented = y_time[start:start+1111]
    return y_segmented

In [149]:
def stft(y_seg , sr, window, step):
    [f,f_name] = sf.feature_extraction(y_seg,sr, window, step,  deltas = False)
    return f

In [150]:
def feature_matricization(y_time):
    y_seg = divide_segments(y_time,0)
    y_stft = stft(y_seg,sr,0.05*sr,0.05*sr)
    y_stft = y_stft.reshape(34)
    
    start = 1111
    y_stfts = y_stft
    while(start <= len(y_time)-1111):
        y_seg = divide_segments(y_time,start)
        #print(y_seg)
        start += 2222
        y_stft = stft(y_seg,sr,0.05*sr,0.05*sr)
        #y_stft_arr = np.asarray(y_stft).T
        y_stft = y_stft.reshape(34)
        y_stfts = np.vstack((y_stfts, y_stft))
        #print(y_stft_arr.shape)

        #np.column_stack((y_stfts, y_stft_arr))
        
    if(len(y_stfts)<70):
        #cal how less
        x = 70 - len(y_stfts)%70
        zeros_arr = np.zeros(shape = (x,34))
        x_arr = np.asanyarray(y_stfts)
        x_arr = x_arr.reshape(-1,34)
        x_final = np.vstack((x_arr,zeros_arr))
        
        return x_final
    if(len(y_stfts)>=70):
        y_stfts = y_stfts[0:70]
        return np.asanyarray(y_stfts)

In [151]:
file_path = paths[0]
label = emotions[0]
labels = np.asarray(label)
y,sr = librosa.load(file_path)
y_time = librosa.frames_to_time(y)
feat = feature_matricization(y_time)
features = feat
for i in range(len(paths)):
    if(i!=0):
        #print(i)
        file_path = paths[i]
        label = emotions[i]
        labels = np.vstack((labels,label))
        y,sr = librosa.load(file_path)
        y_time = librosa.frames_to_time(y)
        feat = feature_matricization(y_time)
        features = np.vstack((features,feat))

In [152]:
type(features)

numpy.ndarray

In [153]:
features.shape

(7700, 34)

In [154]:
indices = features.shape[0]/70
indices

110.0

In [155]:
features_seg = np.split(features,indices)

In [156]:
features_seg

[array([[9.90009083e-02, 1.38312497e-02, 1.73863508e+00, ...,
         1.59910577e-02, 1.32229459e-02, 5.05300354e-03],
        [1.18074478e-01, 5.36852066e-02, 3.16914594e+00, ...,
         1.36317892e-02, 1.42712612e-02, 7.73342336e-03],
        [7.44777475e-02, 4.63305054e-02, 3.27091391e+00, ...,
         5.70295869e-03, 3.19297283e-03, 1.23737758e-02],
        ...,
        [0.00000000e+00, 0.00000000e+00, 0.00000000e+00, ...,
         0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
        [0.00000000e+00, 0.00000000e+00, 0.00000000e+00, ...,
         0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
        [0.00000000e+00, 0.00000000e+00, 0.00000000e+00, ...,
         0.00000000e+00, 0.00000000e+00, 0.00000000e+00]]),
 array([[1.90735695e-02, 1.76300343e-01, 3.25781391e+00, ...,
         1.58075771e-02, 6.06857114e-03, 5.38380868e-02],
        [1.81653043e-02, 2.01037945e-01, 3.30474947e+00, ...,
         1.19805852e-02, 9.65488985e-04, 4.81667269e-02],
        [5.81289737e-02, 

In [157]:
labels

array([['neutral'],
       ['neutral'],
       ['neutral'],
       ['neutral'],
       ['neutral'],
       ['neutral'],
       ['neutral'],
       ['neutral'],
       ['neutral'],
       ['neutral'],
       ['neutral'],
       ['neutral'],
       ['neutral'],
       ['neutral'],
       ['neutral'],
       ['neutral'],
       ['neutral'],
       ['neutral'],
       ['neutral'],
       ['neutral'],
       ['neutral'],
       ['neutral'],
       ['neutral'],
       ['neutral'],
       ['neutral'],
       ['neutral'],
       ['neutral'],
       ['neutral'],
       ['neutral'],
       ['neutral'],
       ['neutral'],
       ['neutral'],
       ['neutral'],
       ['neutral'],
       ['neutral'],
       ['neutral'],
       ['neutral'],
       ['neutral'],
       ['neutral'],
       ['neutral'],
       ['neutral'],
       ['neutral'],
       ['neutral'],
       ['neutral'],
       ['neutral'],
       ['neutral'],
       ['neutral'],
       ['neutral'],
       ['neutral'],
       ['neutral'],


In [158]:
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.utils import to_categorical
le = LabelEncoder()
labels = to_categorical(le.fit_transform(labels))
labels

array([[0., 1.],
       [0., 1.],
       [0., 1.],
       [0., 1.],
       [0., 1.],
       [0., 1.],
       [0., 1.],
       [0., 1.],
       [0., 1.],
       [0., 1.],
       [0., 1.],
       [0., 1.],
       [0., 1.],
       [0., 1.],
       [0., 1.],
       [0., 1.],
       [0., 1.],
       [0., 1.],
       [0., 1.],
       [0., 1.],
       [0., 1.],
       [0., 1.],
       [0., 1.],
       [0., 1.],
       [0., 1.],
       [0., 1.],
       [0., 1.],
       [0., 1.],
       [0., 1.],
       [0., 1.],
       [0., 1.],
       [0., 1.],
       [0., 1.],
       [0., 1.],
       [0., 1.],
       [0., 1.],
       [0., 1.],
       [0., 1.],
       [0., 1.],
       [0., 1.],
       [0., 1.],
       [0., 1.],
       [0., 1.],
       [0., 1.],
       [0., 1.],
       [0., 1.],
       [0., 1.],
       [0., 1.],
       [0., 1.],
       [0., 1.],
       [0., 1.],
       [0., 1.],
       [0., 1.],
       [0., 1.],
       [0., 1.],
       [0., 1.],
       [0., 1.],
       [0., 1.],
       [0., 1.

In [159]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test = train_test_split(features_seg,labels)

In [160]:
len(X_train)

82

In [161]:
len(X_test)

28

In [162]:
len(y_train)

82

In [163]:
len(y_test)

28

In [164]:
y_train.shape

(82, 2)

In [165]:
X_train_arr = np.asanyarray(X_train)
X_train_arr.shape

(82, 70, 34)

In [166]:
X_test_arr = np.asanyarray(X_test)
X_test_arr.shape

(28, 70, 34)

In [167]:
import tensorflow

In [168]:
from tensorflow.keras.layers import Conv1D, MaxPooling1D, Dense, Flatten,Dropout
from tensorflow.keras.models import Sequential

In [169]:
model = Sequential()
model.add(Conv1D(32,5,input_shape = (70,34)))
model.add(Dropout(0.2))
model.add(MaxPooling1D(3))
model.add(Conv1D(64,4))
model.add(MaxPooling1D(3))
model.add(Conv1D(128,5))
model.add(Dropout(0.2))
model.add(MaxPooling1D(2))
model.add(Flatten())
model.add(Dense(256))
model.add(Dense(2,activation = 'sigmoid'))

In [170]:
model.compile(optimizer = 'adam', loss = tensorflow.keras.losses.binary_crossentropy,metrics = ['accuracy'])

In [171]:
model.summary()

Model: "sequential_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv1d_16 (Conv1D)          (None, 66, 32)            5472      
                                                                 
 dropout_8 (Dropout)         (None, 66, 32)            0         
                                                                 
 max_pooling1d_11 (MaxPoolin  (None, 22, 32)           0         
 g1D)                                                            
                                                                 
 conv1d_17 (Conv1D)          (None, 19, 64)            8256      
                                                                 
 max_pooling1d_12 (MaxPoolin  (None, 6, 64)            0         
 g1D)                                                            
                                                                 
 conv1d_18 (Conv1D)          (None, 2, 128)           

In [172]:
model.fit(X_train_arr,y_train,epochs=25, batch_size = 64)

Epoch 1/25
2/2 [==============================] - 1s 11ms/step - loss: 2.3123 - accuracy: 0.5488
Epoch 2/25
2/2 [==============================] - 0s 11ms/step - loss: 1.8442 - accuracy: 0.4878
Epoch 3/25
2/2 [==============================] - 0s 18ms/step - loss: 0.3142 - accuracy: 0.9512
Epoch 4/25
2/2 [==============================] - 0s 9ms/step - loss: 0.6242 - accuracy: 0.7927
Epoch 5/25
2/2 [==============================] - 0s 12ms/step - loss: 0.5657 - accuracy: 0.8415
Epoch 6/25
2/2 [==============================] - 0s 12ms/step - loss: 0.2073 - accuracy: 0.9390
Epoch 7/25
2/2 [==============================] - 0s 13ms/step - loss: 0.1750 - accuracy: 0.9390
Epoch 8/25
2/2 [==============================] - 0s 12ms/step - loss: 0.1779 - accuracy: 0.9634
Epoch 9/25
2/2 [==============================] - 0s 17ms/step - loss: 0.1323 - accuracy: 0.9756
Epoch 10/25
2/2 [==============================] - 0s 13ms/step - loss: 0.0747 - accuracy: 1.0000
Epoch 11/25
2/2 [=============

In [198]:
os.getcwd()

'/Users/aneeshbajwa/Desktop'

In [203]:
model.save(filepath='/Users/aneeshbajwa/Desktop/model.h5')

In [173]:
p = model.predict(X_test_arr)

In [174]:
for i in range(len(p)):
    if(p[i][0]>p[i][1]):
        p[i][0]=1
        p[i][1]=0
    else:
        p[i][0]=0
        p[i][1]=1

In [175]:
p

array([[0., 1.],
       [0., 1.],
       [0., 1.],
       [0., 1.],
       [1., 0.],
       [0., 1.],
       [1., 0.],
       [0., 1.],
       [0., 1.],
       [0., 1.],
       [1., 0.],
       [0., 1.],
       [0., 1.],
       [0., 1.],
       [1., 0.],
       [1., 0.],
       [0., 1.],
       [0., 1.],
       [0., 1.],
       [0., 1.],
       [0., 1.],
       [0., 1.],
       [0., 1.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [0., 1.],
       [0., 1.]], dtype=float32)

In [176]:
model.evaluate(X_test_arr,y_test)

1/1 [==============================] - 0s 169ms/step - loss: 0.2332 - accuracy: 0.8929


[0.23321285843849182, 0.8928571343421936]

In [177]:
#validation

In [178]:
angry_file_paths = glob('../Desktop/Angry/*')
angry_file_paths

['../Desktop/Angry/Recording (10).m4a',
 '../Desktop/Angry/Recording (11).m4a',
 '../Desktop/Angry/Recording (16).m4a',
 '../Desktop/Angry/Recording (7).m4a',
 '../Desktop/Angry/Recording (20).m4a',
 '../Desktop/Angry/Recording (6).m4a',
 '../Desktop/Angry/Recording (17).m4a',
 '../Desktop/Angry/Recording (9).m4a',
 '../Desktop/Angry/Recording (14).m4a',
 '../Desktop/Angry/Recording (5).m4a',
 '../Desktop/Angry/Recording (18).m4a',
 '../Desktop/Angry/Recording (19).m4a',
 '../Desktop/Angry/Recording (4).m4a',
 '../Desktop/Angry/Recording (15).m4a',
 '../Desktop/Angry/Recording (8).m4a',
 '../Desktop/Angry/Recording (3).m4a',
 '../Desktop/Angry/Recording (12).m4a',
 '../Desktop/Angry/Recording (13).m4a',
 '../Desktop/Angry/Recording (2).m4a']

In [182]:
angry_file_path = angry_file_paths[0]
y,sr = librosa.load(angry_file_path)
y_time = librosa.frames_to_time(y)
feat = feature_matricization(y_time)
features_angry = feat
for i in range(len(angry_file_paths)):
    if(i!=0):
        #print(i)
        angry_file_path = angry_file_paths[i]
        y,sr = librosa.load(angry_file_path)
        y_time = librosa.frames_to_time(y)
        feat = feature_matricization(y_time)
        features_angry = np.vstack((features_angry,feat))

In [183]:
features_angry.shape

(1330, 34)

In [184]:
indices_ang = features_angry.shape[0]/70

In [185]:
features_angry_seg = np.split(features_angry, indices_ang)

In [186]:
features_angry_arr = np.asanyarray(features_angry_seg)

In [187]:
features_angry_arr.shape

(19, 70, 34)

In [188]:
p_angry = model.predict(features_angry_arr)

In [189]:
p_angry

array([[9.9411958e-01, 5.5897236e-03],
       [9.7861058e-01, 4.6000481e-03],
       [8.4327769e-01, 1.3909376e-01],
       [8.1225157e-01, 2.4359375e-02],
       [9.9305576e-01, 4.8878193e-03],
       [1.0000000e+00, 6.7733352e-09],
       [6.8661743e-01, 1.9360659e-01],
       [1.0000000e+00, 7.3935843e-18],
       [1.0000000e+00, 9.9900204e-14],
       [1.0000000e+00, 1.1488311e-10],
       [3.3665985e-02, 9.7067821e-01],
       [3.8211364e-01, 5.3810763e-01],
       [7.7065718e-01, 3.6231542e-01],
       [9.6980250e-01, 1.3328850e-02],
       [1.0000000e+00, 8.3384748e-07],
       [1.0000000e+00, 4.7876015e-13],
       [9.9746358e-01, 2.3661256e-03],
       [9.9994242e-01, 3.7801467e-05],
       [9.4966108e-01, 7.7148467e-01]], dtype=float32)

In [190]:
for i in range(len(p_angry)):
    if(p_angry[i][0]>p_angry[i][1]):
        p_angry[i][0]=1
        p_angry[i][1]=0
    else:
        p_angry[i][0]=0
        p_angry[i][1]=1

In [191]:
p_angry

array([[1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [0., 1.],
       [0., 1.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.]], dtype=float32)